In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from database.weather import Weather
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [16]:
## Loading Constants
start = "2008-01-01"
end = datetime(2021,1,7).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("unity")
market = Market()
weather = Weather()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
reload = True
weekly_range = range(1,54)
yearly_range = range(2018,2021)
model_range = range(len(tickers["Symbol"]))
dataset = "pdr"

In [17]:
timeline = DateUtils.create_timeline(start,end)

In [7]:
market.connect()
classification = market.retrieve_data("dataset_{}_week_classification".format(dataset))
regression = market.retrieve_data("dataset_{}_week_regression".format(dataset))
market.close()

In [14]:
for col in regression.columns:
    if -99999 == regression[col].min():
        regression.drop(col,axis=1,inplace=True)

for col in classification.columns:
    if -99999 == classification[col].min():
        classification.drop(col,axis=1,inplace=True)

In [15]:
classification

,_id,year,quarter,week,A,AAL,AAP,AAPL,ABC,ABMD,...,WYNN,XEL,XLNX,XOM,XRAY,XRX,YUM,ZBH,ZBRA,ZION
0,6014124e271422213200b1e4,2008,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6014124e271422213200b1e5,2008,1,2,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,6014124e271422213200b1e6,2008,1,3,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,6014124e271422213200b1e7,2008,1,4,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,6014124e271422213200b1e8,2008,1,5,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,6014124e271422213200b4a8,2020,4,53,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
709,6014124e271422213200b4a9,2021,1,1,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
710,6014124e271422213200b4aa,2021,1,2,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
711,6014124e271422213200b4ab,2021,1,3,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [28]:
year = 2018
week = 1
for year in range(2018,2021):
    for week in range(1,53):
        first = regression[(regression["year"] == year -1) & (regression["week"] == week)].index.values.tolist()[0]
        last = regression[(regression["year"] == year) & (regression["week"] == week)].index.values.tolist()[0]
        print(year,week,regression.iloc[last]["week"])
        

2018 1 1
2018 2 2
2018 3 3
2018 4 4
2018 5 5
2018 6 6
2018 7 7
2018 8 8
2018 9 9
2018 10 10
2018 11 11
2018 12 12
2018 13 13
2018 14 14
2018 15 15
2018 16 16
2018 17 17
2018 18 18
2018 19 19
2018 20 20
2018 21 21
2018 22 22
2018 23 23
2018 24 24
2018 25 25
2018 26 26
2018 27 27
2018 28 28
2018 29 29
2018 30 30
2018 31 31
2018 32 32
2018 33 33
2018 34 34
2018 35 35
2018 36 36
2018 37 37
2018 38 38
2018 39 39
2018 40 40
2018 41 41
2018 42 42
2018 43 43
2018 44 44
2018 45 45
2018 46 46
2018 47 47
2018 48 48
2018 49 49
2018 50 50
2018 51 51
2018 52 52
2019 1 1
2019 2 2
2019 3 3
2019 4 4
2019 5 5
2019 6 6
2019 7 7
2019 8 8
2019 9 9
2019 10 10
2019 11 11
2019 12 12
2019 13 13
2019 14 14
2019 15 15
2019 16 16
2019 17 17
2019 18 18
2019 19 19
2019 20 20
2019 21 21
2019 22 22
2019 23 23
2019 24 24
2019 25 25
2019 26 26
2019 27 27
2019 28 28
2019 29 29
2019 30 30
2019 31 31
2019 32 32
2019 33 33
2019 34 34
2019 35 35
2019 36 36
2019 37 37
2019 38 38
2019 39 39
2019 40 40
2019 41 41
2019 42 42
20